In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import os

In [2]:
train_dir_hotdog = r"/kaggle/input/hotdog-nothotdog/hotdog-nothotdog/hotdog-nothotdog/train/hotdog"
train_dir_nothotdog = r"/kaggle/input/hotdog-nothotdog/hotdog-nothotdog/hotdog-nothotdog/train/nothotdog"
test_dir_hotdog = r"/kaggle/input/hotdog-nothotdog/hotdog-nothotdog/hotdog-nothotdog/test/hotdog"
test_dir_nothotdog = r"/kaggle/input/hotdog-nothotdog/hotdog-nothotdog/hotdog-nothotdog/test/nothotdog"

In [4]:
img_width, img_height = 150, 150
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
def load_and_augment_images(directory):
    images = []
    labels = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            img_path = os.path.join(root, file)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_width, img_height))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)
            images.append(img_array)
            labels.append(1 if "hotdog" in root else 0)
    return np.vstack(images), np.array(labels)

In [ ]:
X_train_hotdog, y_train_hotdog = load_and_augment_images(train_dir_hotdog)
X_train_nothotdog, y_train_nothotdog = load_and_augment_images(train_dir_nothotdog)

X_train = np.concatenate([X_train_hotdog, X_train_nothotdog])
y_train = np.concatenate([y_train_hotdog, y_train_nothotdog])

X_test_hotdog, y_test_hotdog = load_and_augment_images(test_dir_hotdog)
X_test_nothotdog, y_test_nothotdog = load_and_augment_images(test_dir_nothotdog)

X_test = np.concatenate([X_test_hotdog, X_test_nothotdog])
y_test = np.concatenate([y_test_hotdog, y_test_nothotdog])

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

accuracy = model.evaluate(X_test, y_test)[1] * 100
print(f"Accuracy: {accuracy:.2f}%")